# <center>0. Основные определения  


**Модуль** - любой файл с расширением `.py` (даже пустой), а **имя модуля** - название самого файла. Любой модуль в Python может включать в себя переменные, объявления функций и классов. Вдобавок ко всемe, в модуле может содержаться исполняемый код.

**Встроенный модуль** - модуль, который был написан на языке Си, скомпилирован и встроен в интерпретатор Python. Список встроенных модулей зависит от дистрибутива Python, а найти этот список можно в `sys.builtin_module_names`. Как правило туда входят модули `math`, `itertools`, `time` и др. Сам модуль `sys` также является встроенным и информацию о нем можно почитать [здесь](https://pythonworld.ru/moduli/modul-sys.html)   
    
**Модули стандартной библиотеки** - модули, которые не встроены в интерпретатор, но они идут в комплекте с ним, когда мы его скачиваем, например, модули `re`, `os`, `random`. Детальнее про то, какие еще модули туда входят можно почитать в [документации](https://docs.python.org/3/library/index.html)  
    
    
**Пакет** - папка, состоящая из модулей. До Python версии 3.3, чтобы папка с модулями считалась пакетом, она должна была содержать специальный файлик `__init__.py` (Regular Packages). C Python 3.3 это не является обязательным ([Namespace Packages](https://www.python.org/dev/peps/pep-0420/)). Даже такой вырожденный случай как пустая папка тоже является пакетом и его можно импортировать, хотя это бесмысленно). 
    На примере абстрактного пакета обаботки звука, любой пакет на Python имеет приблизительно следующую структуру:  

```
    sound/                          Пакет верхнего уровня
      __init__.py               Инициализирует звуковой пакет
      formats/                  Подпакет для конверсии файловых форматов
              __init__.py
              wavread.py
              wavwrite.py
              auread.py
              auwrite.py
              ...
      effects/                  Подпакет для звуковых эффектов
              __init__.py
              echo.py
              surround.py
              reverse.py
              ...
      filters/                  Подпакет для фильтров
              __init__.py
              equalizer.py
              vocoder.py
              karaoke.py
              ...
    ```
<font color='red'> Важное замечание: <font color='black'> Кроме функции группировки модулей по схожей функциональности ("*мухи отдельно - котлеты отдельно*" или по-умному, **декомпозиция**), пакет еще выполняет очень важную функцию по разделению пространства имен. Разработчикам можно не беспокоиться о том, что в каком-то еще пакете уже есть модуль с названием, который мы хотим использовать для нашего модуля.На примере приведенной выше схемы, мы можем не беспокоиться, что в каком-нибудь пакете `PyAudio` уже есть модуль с названием `wavread.py`, просто потому, что это разные пространства имен и соответственно конфликта в любом случае не будет.
    
**Стороние пакеты (модули)/Пакеты сторонних разработчиков** - это те пакеты (модули), которые не идут в одном комплекте с интерпретатором и мы их должны устанавливать отдельно, например с помощью менеджера пакетов `pip` (`pandas`,`numpy`,`sklearn`).  

<font color='red'>Важное замечание:<font color='black'> Пакеты и модули в Python - это тоже объекты и у них есть свои атрибуты. Причем они оба являются объектами типа `module`. 

# <center> 1. import модуля

### 1.1 Как работает импорт

Если кратко, то импорт модуля работает следующим образом. Работа процедуры импорта модуля включает три шага: **поиск модуля**, **компиляция** и **запуск**.  Поиска модуля, указанного в импорте, интерпретатор Python, выполняет последовательно в ряде директорий, которые заранее заданы + текущая дирректория, если мы импортируем в интерактивном режиме / дирректория скрипта, если запускаем скрипт через командную строку в виде `python \apps\mobile_app\script.py`.  

После того, как модуль был найден, производится его компиляция в байт-код, если это необходимо. Это делается в том случае, если байт-код более старый по сравнению с файлом с исходным кодом (или запускается в другом интерпретаторе). В Python 3 после компиляции создается каталог с именем `__pycache__`, в нем располагаются файлы с расширением `.pyc`, в них содержится байт-код.

После этого следует процедура запуска модуля, которая заключается в последовательном выполнении строк кода в соответствующем модуле. Важно помнить, что модуль импортируется только один раз, при этом, если вы еще раз выполните процедуру импорта, то модуль загружен не будет.

Ну а теперь поподробней обо всем.

 Для наглядности создадим модуль `module.py` и в нем пропишем следующий код
```
   #module.py
    
   print("Importing module.py")
   
   var_1 = 1
   var_2 ="VAR_2"
   
   def foo():
       print("In function foo")
   
   foo()
    
   if __name__ == '__main__':
        print("In __main__  part')
   
```

In [1]:
import module
print('------')
module.foo()

Importing module.py
In function foo
------
In function foo


Мы видим, что код в модуле выполнился при импортировании, а функция `foo()` теперь доступна для применения. Данная функция  теперь нам доступна, поскольку во время исполнения модуль представлен объектом, атрибутами которого являются
   - объявления, присутствующие в файле.
   - объекты, импортированные в этот модуль откуда-либо.  
   
При этом определения и импортированные сущности ничем друг от друга не отличаются: и то, и другое — это всего лишь именованные ссылки на некоторые объекты "первого класса" (такие, которые могут быть переданы из одного участка кода в другой как обычные значения).

Такое единообразие удобно, например, при рефакторинге: мы можем разделить один разросшийся модуль на несколько, а потом импортировать вынесенные определения в оригинальный модуль. При этом с точки зрения внешнего наблюдателя переработанный модуль будет иметь те же атрибуты, которые имел до внесения изменений, а значит у пользователей модуля ничего в коде не сломается.  

Посмотрим на атрибуты нашего модуля `module` с помощью функции `dir(..)`  
 

In [2]:
dir(module)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'foo',
 'var_1',
 'var_2']

Видим, что функция `foo`, переменные `var_1` и `var_2`, которые мы объявили в модуле теперь являются его атрибутами после импортировани и к ним можно обращаться. Кроме них объект модуля имеет еще несколько других атрибутов, которые мы сами не определяли, но они автоматически создаются. Например атрибут `__file__` (содержит полный путь к исходному коду модуля) или атрибут `__package__` (содержит информацию об иерархии пакетов, внутри которых лежит данный модуль `package.subpackage`).

При этом заметим что не выполнился код в блоке `if __name__ == '__main__:`, когда мы импортировали модуль. 

###  1.2  `__name__ ==  '__main__'` ?

В модуле имя модуля (как строка) доступно как значение глобальной переменной `__name__`

In [2]:
module.__name__

'module'

Переменная `__name__` будет иметь значение `__main__`, если модуль запускать из командной строки.
`python module.py <arguments>` и следовательно код, который будет в этом блоке находится, будет выполнен.  

Как правило в блок `if __name__ = '__main__'` помещают тот код, который не должен выполняться при импорте модуля. А при запуске из командной строки он используется, например, для тестирования функционала модуля. 

### 1.3 Повторный импорт модуля

<font color='red'> Важное замечание: <font color='black'> Для повышения эффективности каждый модуль импортируется только однажды за сессию интерпретатора. Если мы попытаемся импортировать уже импортированный модуль еще раз то, интерпретатор сначала проверит его, а не импортирован он уже. И если вдруг окажется, что импортирован, то заново он делать импорт уже не будет. Следовательно, если мы изменяем модуль, то нужно будет перезапустить интерпретатор и заново сделать все импорты, чтобы получить актуальную версию модуля.  
<font color='red'>Аналогично и с пакетами! 

In [1]:
print("First import:")
import module
print("-------------")
print("Second import:")
import module

First import:
Importing module.py
Function foo is declared
Function foo
-------------
Second import:


<font color='black'>Кроме этого, мы можем явно попросить у интерпретатора, чтобы он заново импортировал модуль. Для этого можно воспользоваться функцией `importlib.reload()`:
```
    import importlib; 
    importlib.reload(modulename)
```

In [2]:
import importlib
importlib.reload(module)

Importing module.py
Function foo is declared
Function foo


<module 'module' from 'C:\\Users\\User\\Envs\\diving_in_python\\Diving-in-Python\\Week 1\\module.py'>

### 1.4 `__pycache__` файлы Python  

Для ускорения загрузки модулей Python кэширует скомпилированную в байткод версию каждого модуля в каталоге `__pycache__` под именем `module.version.pyc`, где версия кодирует формат скомпилированного файла; обычно включает номер версии Python. Например, в  моем случае скомпилированная версия модуля `module.py` была кэширована как `__pycache__/module.cpython-35.pyc`. Такое соглашение наименования позволяет компилировать модули из различных релизов и различных версий Python для сосуществования. 

Интерпретатор Python сравнивает дату изменения исходного кода со скомпилированной версией, чтобы обнаружить, если она устарела и нуждается в перекомпиляции. Это полностью автоматический процесс.

Давайте удалим `__pycache__` файл для модуля `module.py`, если он есть, и импортируем этот модуль заново.

In [4]:
import module # будет создан новый __pycache__ файл для модуля module.py
print('Second import:\n')
import importlib # перекомпиляции __pycache__ файла повторной не будет (если конечно мы этот модуль не изменили)
importlib.reload(module)

Second import:

Importing module.py
Function foo is declared
Function foo


<module 'module' from 'C:\\Users\\admin\\Diving-in-Python\\Week 1\\Конспекты\\module.py'>

Также скомпилированные модули не зависят от платформы. Поэтому одна и та же библиотека может быть разделена среди систем с различными архитектурами. Это обеспечивается благодаря тому, что скомпилированный модуль представляет собой байткод, который платформанезависимый.  

<font color='red'> Важное замечание: <font color='black'> Программа не работает быстрее, когда читается из файлов `.pyc`, чем когда она читается из файла `.py`; единственное, что быстрее для файлов `.pyc` - это скорость, с которой они загружаются, потому что отсутствует этап компиляции в байткод, ведь в `.pyc` он там и есть, а в `.py` находится исходный код, который нужно интерпретатору сначала скомпилировать в байткод.
    
Давайте снова вернемся к пакету, который мы рассматривали в самом начале. До комиляции всех, модулей в этом пакете он выглядит следующим образом: 

```
    sound/                          Пакет верхнего уровня
      __init__.py               Инициализирует звуковой пакет
      formats/                  Подпакет для конверсии файловых форматов
              __init__.py
              wavread.py
              wavwrite.py
              auread.py
              auwrite.py
              ...
      effects/                  Подпакет для звуковых эффектов
              __init__.py
              echo.py
              surround.py
              reverse.py
              ...
      filters/                  Подпакет для фильтров
              __init__.py
              equalizer.py
              vocoder.py
              karaoke.py
              ...
```

**После компиляции структура пакета будет примерно следующей:**  

```
    sound/                          Пакет верхнего уровня
      __pycache__
          __init__.cpython-35.pyc
          
      __init__.py               Инициализирует звуковой пакет
      formats/                  Подпакет для конверсии файловых форматов
              __pycache__
                  __init__.cpython-35.pyc
                  wavread.cpython-35.pyc
                  wavwrite.cpython-35.pyc
                  auread.cpython-35.pyc
                  auwrite.cpython-35.pyc
                  
              __init__.py
              wavread.py
              wavwrite.py
              auread.py
              auwrite.py
              ...
      effects/                  Подпакет для звуковых эффектов
              __pycache__
                  __init__.cpython-35.pyc
                  echo.cpython-35.pyc
                  surround.cpython-35.pyc
                  reverse.cpython-35.pyc
     
              __init__.py
              echo.py
              surround.py
              reverse.py
              ...
      filters/                  Подпакет для фильтров
              __pycache__
                  __init__.cpython-35.pyc
                  equalizer.cpython-35.pyc
                  vocoder.cpython-35.pyc
                  karaoke.cpython-35.pyc

              __init__.py
              equalizer.py
              vocoder.py
              karaoke.py
              ...
```

<font color='red'>Важное замечание:<font color='black'> По предыдущей схеме могло сложится впечатление, что при импорте пакета или только модуля из пакета **ДЛЯ ВСЕХ МОДУЛЕЙ** из этого пакета создается скомпилированный файл, но это не так. Скомпилированные файлы будут создаваться **ТОЛЬКО** для модулей, которые **НЕПОСРЕДСТВЕННО** импортируются из пакета. Например, у нас есть пакет со следющей структурой:
    
```
    package/
        sub_package/
            module_1.py
        __init__.py
        module.py

```
 
 После импорта `from package.sub_package import module_1.py` структура будет следующей:
 
 ```
    package/
        __pycache__
            __init__.cpython-35.pyc
            
        sub_package/
            __pycache__
                __init__.cpython-35.pyc
                module_1.cpython-35.pyc
            
            __init__.py
            module_1.py
            
        __init__.py
        module.py

```

видим, что скомпилированный файл создался для модуля `module_1.py`, поскольку мы его импортировали; а для модуля `module.py` не создался, так как мы его явно не импортировали. Кроме этого для модулей может быть создан скомпилированный файл, если они импортируются из файлика `__init__.py`, о котором будет сказано более подробно чуть попозже. Если вкратце, то это тот файлик, который выполняется при импорте пакета, в котором он лежит. В этом файлике могут быть прописаны импорты. Например для пакета `package` этот файл может выглядеть следующим образом:
```
    from . import module # "." говорит, что импорт делать из текущего пакета
```

В этом случае создаться `.pyc` файл не только для файла `__init__.py`, но и для модуля `module.py` 

 ```
    package/
        __pycache__
            __init__.cpython-35.pyc
            module.cpython-35.pyc
            
        sub_package/         
            __init__.py
            module_1.py
            
        __init__.py
        module.py

```

Более подробно по теме `__pycache__` файлов можно почитать в [PEP 3147](https://www.python.org/dev/peps/pep-3147/). Я хотел бы еще добавить следующую наглядную блок-схему, в которой изложена процедура создания скомпилированных файлов.  

<img src="flow_chart.png">

###  1.5 Модули и видимость содержимого

В Python нет настоящего сокрытия атрибутов объектов, поэтому и атрибуты объекта модуля так или иначе всегда доступны после импорта последнего. Однако существует ряд соглашений, которые влияют на процесс импортирования и поведение инструментов, работающие с кодом.

Так атрибуты, имя которых начинается с одиночного подчёркивания, считаются как бы помеченными "для внутреннего использования", и обычно не отображаются в IDE при обращению к объекту "через точку". И linter (статический анализатор кода)  обычно предупреждает об использовании таких атрибутов, мол, "небезопасно!". "Опасность" состоит в том, что автор кода имеет полное право изменять состав таких атрибутов без уведомления пользователей кода. Поэтому программист, использовавший в своём коде приватные части чужого кода рискует в какой-то момент получить код, который перестанет работать при обновлении сторонней библиотеки.

Итак, мы можем определять публичные атрибуты модуля, приватные атрибуты (так называют упомянутые выше атрибуты "для внутреннего пользования"). И данное разделение касается не только определений, содержащихся в самом модуле, но и импортируемых сущностей. Ведь все импортированные объекты становятся атрибутами и того модуля, в который они импортированы.

Есть и третья группа атрибутов — атрибуты, добавляемые в область видимости при импортировании всего содержимого модуля ("со звёздочкой", `from module import *`). Если ничего явно не указывать, то при таком импортировании в текущую область видимости добавятся **ТОЛЬКО РУБЛИЧНЫЕ АТРИБУТЫ МОДУЛЯ**. Помимо данного умолчания существует и возможность явно указать, что конкретное будет экспортировано при импорте со звёздочкой. Для управления названным методом импорта существует специальный атрибут `__all__`, в который можно положить **СПИСОК СТРОК С ИМЕНАМИ**, которые будут экспортироваться **И ТОЛЬКО ОНИ**. 


Рассмотрим пример, демонстрирующий всё вышеописанное.

**Пример:** 


Пусть у нас будет два файла:

```
# Файл "module.py"
from other_module import CAT, DOG as _DOG, _GOAT

FISH = 'fish'
MEAT = 'meat'
_CARROT = 'carrot'

__all__ = ('FISH', '_CARROT')
```

и файл  

```
# Файл "other_module.py"
CAT = 'cat'
DOG = 'dog'
_GOAT = 'goat'
```

Рассмотрим сначала обычный импорт `import module`. Если импортировать модуль таким образом, то IDE, REPL и остальные инструменты "увидят" у модуля следующие атрибуты через автодополнение(. + TAB):

`FISH`, `MEAT` т.к. имена констант — публичные,
`CAT`, т.к. константа импортирована под публичным именем.

А эти атрибуты не будут видны:

 - `_DOG`, т.к. при импортировании константа переименована в приватной манере, 
 - `_GOAT`, т.к. импортирована по своему приватному имени (тут линтер может и поругать за обращение к приватному атрибуту модуля!),
 - `_CARROT`, ибо приватная константа.
 
Но это вовсе не означает, что мы не можем обратиться к приватным атрибутам модуля. Просто нужно будет перед нажатием клавиши TAB напечатать нижнее подчеркивание точки ".\_" и нажать TAB и тогда мы сможем увидеть приватные атрибуты модуля и обратиться к ним. Либо если нас интересует атрибут, который мы знаем как называется, то можем вручную без автодополнения напечатать. Например, давайте импортируем модуль `module.py` и выведем значение атрибута `_DOG`



In [2]:
import module
print(module._DOG)

dog


Импорт `import other_module` я не рассматриваю как тривиальный случай.

Теперь рассмотрим импорт всего содержимого `module`: `from module import *`

После импортирования в текущей области видимости мы получим ровно два новых имени:  `FISH` и `_CARROT` — именно они перечислены в атрибуте `__all__`. Заметьте, что в данном случае при массовом импорте добавится даже приватный атрибут, потому что он явно указан! При этом даже публичные в модуле `module.py` атрибуты `CAT`, `MEAT` не будут импортированны. 

Последствия импорта `from other_module import *` тоже очевидны и я их не рассматриваю.

<font color='red'> ВАЖНОЕ ЗАМЕЧАНИЕ: <font color='black'> Не рекомендуется использовать `*` в импортах, потому что во-первых импортируется не только то, что будет использоваться, но и множество неиспользуемых объектов, что засоряет память.  И ,как следствие, во-вторых, могут возникнуть конфликты в пространстве имен из-за того, что программист в явном виде не видит, какие имена у импортируемых объектов через `*`.
    
<font color='red'> ВАЖНОЕ ЗАМЕЧАНИЕ: <font color='black'> Импорты в Python являются регистрозависимыми, т.е `import module` - это не то же самое, что и импорт `import Module` 

# <center> 2. import пакета  

### 2.1 Общая идея и файлик `__init__`

При импорте пакета Python выполняет код в файле пакета `__init__.py`, если такой имеется. Все объекты, определённые в модуле или `__init__.py`, становятся доступны импортирующему.  Начиная с версии 3.3 этот инициализирующий модуль необязателен для того, чтобы дирректория считалась пакетом. В этом случае мы будем иметь дело с Namespace пакетом, но он по-прежнему будет является объектом типа `module`. 


В `__init__` файле как правило можно увидеть код, который выполняет следующее: 
  - проверка наличия зависимостей для пакета
  - определение объектов, доступных из этого пакета (`__all__`)
  - какие импорты еще делать при импорте данного пакета
--- 

Для примера ниже приведен пример `__init__` файла для пакета `pandas`
<img src="pandas_init.png">

<font color='red'>ВАЖНОЕ ЗАМЕЧАНИЕ: <font color='black'>Прочие модули пакета и вложенные пакеты не импортируются автоматически вместе с пакетом-родителем, но могут быть импортированы отдельно с указанием полного имени.
    Например, у нас пакет со следующей структурой:
```
.
└── package/
    ├── __init__.py
    ├── module.py
    └── subpackage/
        ├── __init__.py
        └── submodule.py
```
    
Если мы напишем `import package`, то загрузится и выполнится только модуль `__init__` и все. Нам не будет доступен модуль `module.py` (если он конечно не импортирован внутри самого файлика `__init__`), нам не будет импортирован дочерний пакет `subpackage` и тем более его модули.

<font color='red'>ВАЖНОЕ ЗАМЕЧАНИЕ: <font color='black'> При импортировании вложенного модуля всегда сначала импортируются модули инициализации всех родительских пакетов (если оные ещё ни разу не импортировались).  


**Пример:**

Например, у нас пакет со следующей структурой:  

```
.
└── package/
    ├── __init__.py
    ├── module.py
    └── subpackage/
        ├── __init__.py
        └── submodule.py
```

1. Когда мы импортируем модуль `submodule.py`:
   - `from package.subpackage import submodule`  
   - `import package.subpackage.submodule`    

   то фактически происходит следующее (именно в таком порядке):

    - загружается и выполняется модуль `package/__init__.py`,
    - загружается и выполняется `package/subpackage/__init__.py`,
    - наконец, импортируется `package/subpackage/submodule.py`.  
   
   
2. При импорте `module` из пакета `package`:

   - `from package import module`  
   - `import package.module`  

   последовательность следующая:  
    
   - загружается модуль `package/__init__.py`.
   - импортируется `package/module.py`

---

Также не забываем про то, что еще будут создаваться `__pycache__` файлы, если его еще не было создано для модуля или он требует перекомпиляции ввиду изменения исходного кода модуля. 

После пункта 1 может сложится впечатление что, если мы загрузим парочку вложенных модулей, то для каждого будет выполняться загрузка всех `__init__.py` по дороге? Не будет! Как ранее уже упоминалось, подсистема интерпретатора, отвечающая за загрузку модулей, кэширует уже загруженные пакеты и модули. И поэтому каждый конкретный модуль загружается ровно один раз, в том числе и инициализирующие модули `__init__.py`.

**Пример:**  

Пусть имеется следующая структура пакета:  

```
.
└── package/
    ├── __init__.py
    ├── module.py
    └── subpackage_1/
        ├── __init__.py
        └── submodule_1.py
    └── subpackage_2/
        ├── __init__.py
        └── submodule_2.py
```

Мы выполняем следующие импорты:   
   
   - `from package.subpackage_1 import submodule_1`
   - `from package.subpackage_2 import submodule_2`

В первом случае шаги будут следующие:  
   
   - загружается и выполняется модуль `package/__init__.py`,
   - загружается и выполняется `package/subpackage_1/__init__.py`,
   - наконец, импортируется `package/subpackage_1/submodule_1.py`. 
   
Во втором случае мы увидим следующее:  
   
   - модуль `package/__init__.py` загружаться уже <font color='red'>НЕ БУДЕТ<font color='black'>, поскольку он уже был загружен при прошлом импорте
   - загружается и выполняется `package/subpackage_2/__init__.py`,
   - наконец, импортируется `package/subpackage_2/submodule_2.py`.
    
---

### 2.2 Зачем нужен атрибут  `_all__` для пакетов?

В целом атрибут `__all__` в модуле инициализации пакета (в файлике `__init__.py`) ведёт себя так же, как и в случае с обычным модулем. Но если при импорте пакета "со звёздочкой" среди перечисленных имён встретится имя вложенного модуля, а сам модуль не окажется импортирован ранее в этом же `__init__.py`, то этот модуль импортируется неявно! Очередной пример это продемонстрирует.

**Пример:**

Вот структура пакета:
```
    .
└── package/
    ├── __init__.py
    ├── a.py
    └── b.py
```

Файл же `package/__init__.py` содержит следующее (и только это!):

```
__all__ = ('a', 'b')
```

А импортируем мы `from package import *`. В области видимости у нас окажутся объекты модулей `a` и `b` под своими именами (без полного пути, то есть не нужно писать путь ввида `package.a.` как мы бы это сделали, если просто сделали `import package.a`). При этом сами модули в коде нигде явно не импортируются! Такая вот "автомагия".

---


Указанный автоматизм достаточно ограничен: не работает "вглубь", например — не импортирует "через звёздочку" указанные модули и подпакеты. Если же вам вдруг такого захочется, вы всегда сможете на соответствующих уровнях в `__init__.py` сделать `from x import *` и получить в корневом пакете плоскую область видимости со всем нужным содержимым. Но такое нужно довольно редко, потому что "не помогает" ни IDE, ни ручному поиску по коду. Впрочем, знать о фиче и иметь её ввиду — не вредно, как мне кажется.

**Пример:**

Вот структура пакета:
```
    .
└── package/
    ├── __init__.py
    ├── a.py
    └── b.py
    └── subpackage
        └── __init__.py
        └── submodule.py
```

Файл `package/__init__.py` содержит следующее (и только это!):

```
__all__ = ('subpackage', 'b')
```

А импортируем мы `from package import *`. В этом случае импортируются модуль `b.py` и пакет `package`, а вот `submodule.py` не импортируется.

Если нам все же нужно, чтобы импортнулся и модуль `submodule.py`, то можем написать в инициализирующем файле импорт `submodule.py` и тогда он тоже будет нам доступен как атрибут модуля `subpackage`, а именно: 


Файл `package/subpackage/__init__.py` должен содержать следующее:

```
from . import submodule       #вариант import submodule работать не будет

```

Может возникнуть идея, а давайте в `__init__` файле подпакета `subpackage` будем выполнять импорт модуля `submodule.py` используя атрибут `__all__` и она как раз нам импортирует нужные модули и вернет только их (то есть запустим рекурсионный процесс получения всех подмодулей и подпакетов), но этот вариант не сработает, поскольку импорт через `*` осуществляет импорт вида `from . import <package in __all__/module in __all__>` для всех пакетов и модулей, которые там прописаны, а значит `__all__` в подпакетах/подмодулях не будет срабатывать и соответственно не импортируется нужный нам модуль `submodule.py`. Про знак `.` в импорте будет рассказано чуть попозже.  

### 2.3 Циклические импорты

Если вдруг вы захотите в один модуль импортировать другой, а другой захочет, пусть даже и не напрямую, импортировать первый, то вы получите ImportError. Потому что у вас случится циклический импорт. Про оный нужно просто знать и стараться архитектурить код так, чтобы циклов не случалось.

Если же приспичивает, и импортировать что-то "ну очень нужно", то можно попробовать обойтись локальным импортом:

```
### foo.py
import bar

A = bar(42)



### bar.py
# import foo — тут не импортируем

def bar(x):
    return x + 1

def baz():
    import foo  # а тут уже можно (локальный импорт), но это огромный КОСТЫЛЬ, поэтому старайтесь избегать таких импортов
    return foo.A
```

# <center> 3. Поиск пакетов и модулей

При импорте модуля интерпретатор сначала ищёт встроенный модуль с таким именем либо папку, если мы говорим про пакеты. Если такого модуля нет, то идёт поиск в списке директорий, определённых в переменной `sys.path`. `sys.path` инициализируется из следующих мест ([Python 3](https://docs.python.org/3/tutorial/modules.html#the-module-search-path)):
   - директории, содержащей исходный скрипт (если через командную строку запускаем скрипт `python script.py`) или текущей директории, если просто запускаем интерпретатор в интерактивном режиме без запуска скрипта (текущая директория - папка из которой был запущен интерпретатор);
   - директории по умолчанию, которая зависит от дистрибутива Python;
   - PYTHONPATH (список имён директорий; имеет синтаксис, аналогичный переменной окружения PATH).

Пайтон ищет модули и пакеты в директориях в списке `sys.path` по порядку в списке от первого пути к последнему. В этом списке пути до модулей стандартной библиотеки обычно расположены раньше, чем директории со сторонними пакетами, чтобы нельзя было случайно заменить стандартный пакет сторонним (среди нескольких с одинаковыми именами загружается первый попавшийся пакет).

В списке путей (обычно в начале) присутствует и путь `' '`, означающий текущую директорию. Это, в свою очередь, означает, что модули и пакет в текущем проекте имеют больший приоритет.  

Обычно пути трогать не нужно, всё вполне нормально "работает само". Но если очень хочется, то путей несколько:

  - Использовать переменную окружения `PYTHONPATH` (значение — строка с путями, разделёнными символом :),
  - Во время исполнения изменить `sys.path`.  
  
**Первый способ** — простой и понятный. Не сложнее добавления пути до исполняемых файлов в PATH (даже синтаксис тот же).

**Второй способ** — сложный и требующий внимательности. Дело в том, что `sys.path` нужно изменять максимально рано — где-нибудь в точке входа. Если не торопиться менять `sys.path`, то что-то уже может успеть загрузиться до того, как вы перестроите пути для поиска пакетов. А ведь эта загрузка может произойти в другом потоке исполнения! Отлаживать проблемы с очерёдностью загрузки модулей сложно. Лучше просто их не создавать.

Есть модуль `pkgutil`, который по пути позволяет посмотреть, какие модули и пакеты там хранятся
```
import pkgutil
search_path = [<path>]
all_modules = [x[1] for x in pkgutil.iter_modules(path=search_path)]
print(all_modules)
```

# <center> 4. Использование объектов из импортированного модуля или пакета  
    
Есть 4 разных вида импортов:
  - `import <пакет>`
  - `import <модуль>`
  - `from <пакет> import <модуль или подпакет или объект>`
  - `from <модуль> import <объект>`
  
Пусть `X` — имя того, что идёт после import:
    
  - `X` — имя модуля или пакета, то для того, чтобы использовать объекты, определённые в X, придётся писать `X.объект`.
  - `X` — имя переменной, то её можно использовать напрямую.
  - `X` — имя функции, то её можно вызвать с помощью `X()`
  
Про область видимости при импорте я писал выше и не стоит про нее забывать.

<font color='green'>Полезно знать следующее:   
  - <font color='black'>Опционально после любого выражения `import X` можно добавить `as Y`. Это переименует `X` в `Y` в пределах скрипта. Учтите, что имя `X` с этого момента становится недействительным. Частым примером такой конструкции является `import numpy as np`.
   
   
  - <font color='black'> Аргументом для `import` может быть как одно имя, так и их список. Каждое из имён можно переименовать с помощью `as`. Например, следующее выражение будет корректным: `import packA as pA, packA.a1, packA.subA.sa1 as sa1`
    
<font color='black'> Ну и последнее про что я хотел бы рассказать - это **абсолютный** и **относительный** импорт.

# <center> 5. Абсолютный и относительный импорт  
    
При **абсолютном импорте** используется полный путь (от начала корневой папки проекта) к желаемому модулю.

При **относительном импорте** используется относительный путь (начиная с пути текущего модуля) к желаемому модулю. Есть два типа относительных импортов:

  - При явном относительном импорте используется формат `from .<модуль/пакет> import X`, где символы точки `.` показывают, на сколько директорий «вверх» нужно подняться. Одна точка `.` показывает текущую директорию, две точки `..` — на одну директорию выше и т. д.
  
  
  - Неявный относительный импорт пишется так, как если бы текущая директория была частью `sys.path`. Такой тип импортов поддерживается только в Python 2.
  
  
В документации Python об относительных импортах в Python 3 написано следующее:

Единственный приемлемый синтаксис для относительных импортов — `from .[модуль] import [имя]`. Все импорты, которые начинаются не с точки `.`, считаются абсолютными. Рассмотрим следующий пример:

**Пример:**

Пусть имеется следующая структура пакета: 

```
.

└── package/
    ├── __init__.py
    └── b.py
    ├── main.py
    └── subpackage
        └── __init__.py
        └── submodule.py
```

В качестве примера допустим, что мы запускаем скрипт `main.py`, который импортирует `package`, модуль `b`, пакет `subpackage` и модуль `submodule` . Тогда импорты в `main.py` будут выглядеть следующим образом:

**Абсолютные импорты:**
```
import package.b
import package.subpackage.submodule

```
**Явные относительные импорты:**
```
from . import b
from .. import package
from . import subpackage
from .subpackage import submodule
```

Неявные относительные импорты (не поддерживаются в Python 3):

import other
import a2
import subA.sa1
Учтите, что в относительных импортах с помощью точек . можно дойти только до директории, содержащей запущенный из командной строки скрипт (не включительно).